# Import / Config

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from neo4j import GraphDatabase
import cred as c

## Postgres

In [2]:
engine = create_engine(
    f'postgresql://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}', 
    connect_args = {'options': '-c search_path=usozmed,public', 'keepalives_idle': 120},
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [3]:
# engine.dispose()

In [4]:
sql = """
SELECT count(*)
FROM 
  Person p1
  join Person_knows_Person pkp1 on pkp1.person1id = p1.id
  join Person p2 on pkp1.person2id = p2.id
  join Person_knows_Person pkp2 ON pkp2.Person1Id = p2.id
  join Person p3 on pkp2.person2id = p3.id
  join Person_hasInterest_Tag phit ON p3.id = phit.personid
  left join Person_knows_Person pkp3 on pkp3.person1id = p3.id and pkp3.person2id = p1.id
 where p3.id!=p1.id and  pkp3.person2id is null;
"""
with engine.connect() as con:
    df = pd.read_sql_query(text(sql), con)
df

In [5]:
df

,count
0,176011014


## Person

In [8]:
cypher = """
MATCH (person1:Person)-[:KNOWS]->(person2:Person)-[:KNOWS]->(person3:Person)-[:HAS_INTEREST]->(tag:Tag)
WHERE NOT (person1)<-[:KNOWS]-(person3)
AND person1 <> person3
RETURN count(*) AS count
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(cypher)
        df = pd.DataFrame(result.data())
df

,count
0,176011014


In [9]:
cypher = """
MATCH (person1:Person)
RETURN count(*) AS count
"""
with GraphDatabase.driver(c.neo4j_host, auth=(c.neo4j_userid, c.neo4j_password)) as driver:
    with driver.session() as session:
        result = session.run(cypher)
        df = pd.DataFrame(result.data())
df

,count
0,10295
